In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric

In [ ]:
pip install "git+https://github.com/tqdm/tqdm.git@devel#egg=tqdm"

In [5]:
pip install ogb

**Loading data and split into source and target(node_year_dic issue solved)**

In [ ]:
import torch
import torch.nn.functional as F
import torch_geometric
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import Data
from torch_geometric.utils import subgraph

# ---------------------------------------------------
# DEVICE
# ---------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# ---------------------------------------------------
# Data
# ---------------------------------------------------

# LOAD Arxiv
arxiv_dataset = PygNodePropPredDataset(name='ogbn-arxiv')
arxiv_data = arxiv_dataset[0]

arxiv_data = T.ToSparseTensor()(arxiv_data)
##arxiv_data.adj_t = arxiv_data.adj_t.to_symmetric()
arxiv_data = arxiv_data.to(device)

arxiv_split_idx = arxiv_dataset.get_idx_split()
arxiv_evaluator = Evaluator(name='ogbn-arxiv')
# ---------------------------------------------------

# LOAD MAG
dataset = PygNodePropPredDataset(name="ogbn-mag")
rel_data = dataset[0]
#print(dir(rel_data))
#print(rel_data)



data = Data(
    x=rel_data.x_dict['paper'],
    edge_index=rel_data.edge_index_dict[('paper', 'cites', 'paper')],
    y=rel_data.y_dict['paper']
).to(device)

# SPLIT INTO SOURCE & TARGET SET
years = rel_data.node_year['paper'].unique()
source_years = years[:5]
target_years = years[5:]

source_nodes = torch.cat([
                    torch.where(rel_data.node_year['paper'] == year)[0]
                    for year in source_years
                ])

target_nodes = torch.cat([
                    torch.where(rel_data.node_year['paper'] == year)[0]
                    for year in target_years
                ])

source_nodes, _ = source_nodes.sort()
target_nodes, _ = target_nodes.sort()

source_edge_index, _ = subgraph(source_nodes, data.edge_index, relabel_nodes=True)
target_edge_index, _ = subgraph(target_nodes, data.edge_index, relabel_nodes=True)

source_data = Data(
                x=rel_data.x_dict['paper'][source_nodes],
                edge_index=source_edge_index,
                y=rel_data.y_dict['paper'][source_nodes]
            )

target_data = Data(
                x=rel_data.x_dict['paper'][target_nodes],
                edge_index=target_edge_index,
                y=rel_data.y_dict['paper'][target_nodes]
            )

data = target_data.to(device) # Train on Target split

# MAG EVALUATOR
evaluator = Evaluator(name="ogbn-mag")
# ---------------------------------------------------

**define training and pretraining model for MAG and Arxiv**

In [ ]:
import torch
from torch import optim, nn
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GINConv
from models import *
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from tqdm import tqdm
from copy import deepcopy
import numpy as np


def train(model, optimiser, data):
    # TRAIN
    model.train()
    optimiser.zero_grad()

    try:
        out = model(data.x, data.adj_t)
    except:
        out = model(data.x, data.edge_index)

    loss = F.nll_loss(out, data.y.squeeze(1))

    loss.backward()
    optimiser.step()

    # EVAL
    y_pred = out.argmax(dim=-1, keepdim=True)
    acc = evaluator.eval({
            'y_true': data.y,
            'y_pred': y_pred,
        })['acc']

    return loss.item(), acc


def pretrain_mag_source(model, optimiser, data, model_name, epochs=1000):
    best_acc = 0.0

    for epoch in tqdm(range(epochs)):
        # TRAIN
        model.train()
        optimiser.zero_grad()

        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out, data.y.squeeze(1))

        loss.backward()
        optimiser.step()

        # EVAL
        y_pred = out.argmax(dim=-1, keepdim=True)
        acc = evaluator.eval({
                'y_true': data.y,
                'y_pred': y_pred,
            })['acc']

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), 'source/{}_mag_source.pth'.format(model_name))

    return best_acc



**pretraining on source split MAG GCN**

In [ ]:
# PRETRAIN ON SOURCE SPLIT
from models import *

model = GCN(
        in_channels=data.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        num_layers=3
    ).to(device)


model.reset_parameters()
source_optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
print('Pretraining model on MAG Source split')
best_acc = pretrain_mag_source(model, source_optimiser, source_data.to(device), "GCN")
print('Best accuracy: {:.3}'.format(best_acc))
model.load_state_dict(
        torch.load( '/content/source/{}_mag_source.pth'.format("GCN") )
    )



**Training on Target MAG using Pretrained source MAG(Fine Tuning)**

In [ ]:
# ---------------------------------------------------
    # MODEL
    # ---------------------------------------------------
    # USE MAG MODEL
model = GCN(
        in_channels=data.num_features,
        hidden_channels=253,
        out_channels=dataset.num_classes,
        num_layers=3
    ).to(device)

#MAG TARGET TRAINING
print('Training on MAG')
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(2)):
  train_loss, acc = train(model, optimiser, data)
  print('Epoch: {:03d}, Train Loss: {:.3f}, Train Acc: {:.3f}'.format(epoch, train_loss, acc))

**Feature Extraction**

step 1: split target into train test and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

# Ensure that indices are in the correct format
target_indices = torch.arange(target_data.num_nodes)

# Split indices into train (80%) and temp (20%)
train_indices, temp_indices = train_test_split(target_indices, test_size=0.2, random_state=42)

# Split temp into validation (10%) and test (10%)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

train_indices = torch.tensor(train_indices, dtype=torch.long).to(device)
val_indices = torch.tensor(val_indices, dtype=torch.long).to(device)
test_indices = torch.tensor(test_indices, dtype=torch.long).to(device)

print(f'Train indices: {train_indices.size(0)}')
print(f'Validation indices: {val_indices.size(0)}')
print(f'Test indices: {test_indices.size(0)}')


step 2: feature extraction using the pretrained model

In [ ]:
def extract_features(model, data):
    model.eval()
    with torch.no_grad():
        features = model.convs[0](data.x, data.edge_index)
        for conv in model.convs[1:]:
            features = conv(features, data.edge_index)
            features = F.relu(features)
        return features

def train_classifier(features, labels, train_idx, val_idx, test_idx, num_classes):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score

##ajust iterations
    clf = LogisticRegression(max_iter=100)
    clf.fit(features[train_idx.cpu()], labels[train_idx.cpu()].squeeze(1))

    val_preds = clf.predict(features[val_idx.cpu()])
    test_preds = clf.predict(features[test_idx.cpu()])

    val_acc = accuracy_score(labels[val_idx.cpu()].cpu(), val_preds)
    feat_test_acc = accuracy_score(labels[test_idx.cpu()].cpu(), test_preds)

    return val_acc, test_acc

# Load the pretrained model
model.load_state_dict(torch.load('/content/source/GCN_mag_source.pth'))

# Extract features from the target data using the pretrained model
target_features = extract_features(model, target_data)

# Define the number of classes
num_classes = target_data.y.max().item() + 1

# Train a classifier on the extracted features
feat_val_acc, feat_test_acc = train_classifier(target_features.cpu(), target_data.y.cpu(), train_indices, val_indices, test_indices, num_classes)

print(f'Validation Accuracy: {val_acc:.4f}, Test Accuracy: {feat_test_acc:.4f}')


**Baseline GCN training without transfer learning**

In [ ]:
def train_baseline_gcn(model, optimizer, data, train_idx, val_idx, epochs=10):
    best_val_acc = 0.0

    for epoch in tqdm(range(epochs)):
        # TRAIN
        model.train()
        optimizer.zero_grad()

        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[train_idx], data.y[train_idx].squeeze(1))

        loss.backward()
        optimizer.step()

        # EVAL
        model.eval()
        with torch.no_grad():
            out = model(data.x, data.edge_index)
            val_loss = F.nll_loss(out[val_idx], data.y[val_idx].squeeze(1)).item()
            val_acc = (out[val_idx].argmax(dim=1) == data.y[val_idx].squeeze(1)).sum().item() / val_idx.size(0)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'baseline/gcn_baseline.pth')

        print(f'Epoch {epoch:03d}, Loss: {loss.item():.4f}, Val Acc: {val_acc:.4f}')

    return best_val_acc

def evaluate_model(model, data, test_idx):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        #will change test_acc name to base_test_acc
        base_test_acc = (out[test_idx].argmax(dim=1) == data.y[test_idx].squeeze(1)).sum().item() / test_idx.size(0)
    return test_acc


In [ ]:
# Use predefined target MAG dataset split
target_train_indices = train_indices.to(device)
target_val_indices = val_indices.to(device)
target_test_indices = test_indices.to(device)

# Initialize the model, optimizer, and other parameters
baseline_model = GCN(in_channels=128, hidden_channels=253, out_channels=num_classes, num_layers=3, dropout=0.5).to(device)
baseline_optimizer = torch.optim.Adam(baseline_model.parameters(), lr=0.01, weight_decay=5e-4)

# Train the baseline GCN model
best_val_acc = train_baseline_gcn(baseline_model, baseline_optimizer, target_data, target_train_indices, target_val_indices, epochs=2)

print(f'Best Validation Accuracy (Baseline GCN): {best_val_acc:.4f}')

# Load the best model
baseline_model.load_state_dict(torch.load('baseline/gcn_baseline.pth'))

# Evaluate the model on the test set
base_test_acc = evaluate_model(baseline_model, target_data, target_test_indices)

print(f'Test Accuracy (Baseline GCN): {base_test_acc:.4f}')


**Comparison**

In [ ]:
# Compare the models
print("Baseline Model:")
print(f"Test Accuracy: {base_test_acc:.4f}")

print("Fine-Tuned Model:")
# Assume fine_tune_val_acc and fine_tune_test_acc are obtained during fine-tuning
#print(f"Validation Accuracy: {fine_tune_val_acc:.4f}, Test Accuracy: {fine_tune_test_acc:.4f}")
print(f"Test Accuracy: {acc:.4f}")

print("Feature Extraction Model:")
print(f" Test Accuracy: {feat_test_acc:.4f}")
